In [ ]:
import pandas as pd
import numpy as np
import os
os.chdir("..")
     

DATA = "./data/preprocessed.json"
MODEL = "bert-base-uncased"

RESPONSE_1 = "1. Your company/vision/mission in a tweet ! *"
RESPONSE_2 = "2. Which sector/subsector of Fintech do you operate in?"
RESPONSE_3 = "3. SStage of company *"
RESPONSE_4 = "4. Which problem does your company solve? *"
RESPONSE_5 = "5. What sparked the founder(s) to set up the company? *"
RESPONSE_6 = "6. Please describe the business model of your company, including the main sources of revenue. *"
RESPONSE_7 = "7. What is your USP versus traditional competitors or those in the digital world? *"
RESPONSE_8 = "8. Which target group(s) in which markets do you address primarily? *"
RESPONSE_9 = "9. Please share any other office locations, you might have!"
RESPONSE_10 = "10. Are you a German startup?"
RESPONSE_11 = "11. If you are a foreign startup: Do you have clients in Germany? How many? In B2C/B2B?"
RESPONSE_12 = "12. Please provide a brief competition landscape for your core markets. *"
RESPONSE_13 = "13. Did you pivot in the past and why?"
RESPONSE_14 = "14. Which business area in the last year consumed most of your management attention? *"
RESPONSE_15 = "15. What protects your business from new market entrants or copycats? *"
RESPONSE_16 = "16. After your next financing round, will the founders jointly hold above or below 50% of the"
RESPONSE_17 = "17. Anything else you would want to add or let us know?"
RESPONSE_18 = "18. How do you integrate ESG standards in your business model? *"
RESPONSE_19 = "19. When scaling the company, how do you ensure that the corporate culture does not suffer and"
RESPONSE_20 = "20. Please upload your startup logo. *"
RESPONSE_21 = "21. Did your firm apply to FTGA before? If yes, which year? *"

In [ ]:
df = pd.read_json(DATA)
df = df[['Filename', RESPONSE_1, 'label']]
df[RESPONSE_1] = df[RESPONSE_1].str.lower()

text = df[RESPONSE_1].values
labels = df['label'].values

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(MODEL, do_lower_case=True)
# Bestimmung der Maximallänge der Sätze, um Platz zu sparen
max_len = max([len(tokenizer.encode(t, add_special_tokens=True)) for t in text])
max_len

In [ ]:
import torch
# Jetzt alle Sätze tokenisieren und IDs merken
input_ids = []
attention_masks = []

for t in text:
    encoded_dict = tokenizer.encode_plus(
                        t,
                        add_special_tokens = True,    # '[CLS]' und '[SEP]'
                        truncation = True,
                        max_length=128,
                        pad_to_max_length = True,
                        return_attention_mask = True,  # Attention-Masks erzeugen
                        return_tensors = 'pt',         # pytorch-Tensoren als Ergebnis
                   )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

# Python-Listen in Tensoren wandeln
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Headline, Tokenisierung und IDs anzeigen
print(text[0])
print(tokenizer.tokenize(text[0]))
print(input_ids[0])

In [ ]:
from torch.utils.data import TensorDataset, random_split

# Wir arbeiten ab jetzt nur noch mit dem Input-Tensor, der Attention Mask und den Labeln
dataset = TensorDataset(input_ids, attention_masks, labels)

# wir nutzen einen 3:1-Split für Training und Validierung
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print(train_size, val_size)

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# die BERT-Autoren empfehlen für Finetuning Batch-Sizes von 16 oder 32
batch_size = 16

# DataLoader fÃ¼r die beiden Datensets erzeugen (man könnte auch RandomSampler verwenden)
train_dataloader = DataLoader(train_dataset, sampler = RandomSampler(train_dataset), batch_size = batch_size)
validation_dataloader = DataLoader(val_dataset, sampler = SequentialSampler(val_dataset), batch_size = batch_size)

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# das Modell muss zum Tokenizer passen!
model = BertForSequenceClassification.from_pretrained(
    MODEL, 
    num_labels = 7, # wir haben nur gut oder shlecht
    output_attentions = False,
    output_hidden_states = False # wir benÃ¶tigen keine Embeddings
)
# hier evtl. model.cpu() einsetzen
model.cuda()

In [ ]:
# Optimierer auswählen, AdamW ist Standard
optimizer = AdamW(model.parameters(), lr = 2e-5)

In [ ]:
from transformers import get_linear_schedule_with_warmup

# vier Epochen, das kann justiert werden
epochs = 32
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)

In [ ]:
import numpy as np

# Accuracy berechnen
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import random
import numpy as np
from tqdm.auto import trange, tqdm
import torch

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print("Using GPU %s" % torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("Using CPU :-(")

# Statistik fÃ¼r das Training
training_stats = []

for epoch_i in trange(epochs, desc="Epoche"):
    # akkumulierter Loss für diese Epoche
    total_train_loss = 0

    # Modell in Trainingsmodus stellen
    model.train()

    # Trainig pro Batch
    for step, batch in enumerate(tqdm(train_dataloader, desc="Training")):
        # Daten entpacken und in device-Format wandeln
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Gradienten löschen
        model.zero_grad()        

        # Vorwärts-Auswertung (Trainingsdaten vorhersagen)
        res = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        # Loss berechnen und akkumulieren
        total_train_loss += res.loss.item()

        # Rückwärts-Auswertung, um Gradienten zu bestimmen
        res.loss.backward()

        # Gradient beschrÃ¤nken wegen Exploding Gradient
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Parameter und Lernrate aktualisieren
        optimizer.step()
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
      

    # Modell in Vorhersage-Modus umstellen
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Eine Epoche validieren
    for batch in tqdm(validation_dataloader, desc="Validierung"):
        # jetzt die Validierungs-Daten entpacken
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Rückwärts-Auswertung wird nicht benötigt, daher auch kein Gradient
        with torch.no_grad():        
            # Vorhersage durchfÃ¼hren
            res = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        # Loss akkumulieren
        total_eval_loss += res.loss.item()

        # Vorhersagedaten in CPU-Format wandeln, um Accuracy berechnen zu können
        logits = res.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Accuracy für die Verifikation
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    tqdm.write("Accuracy: %f" % avg_val_accuracy)

    # Loss über alle Batches
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    tqdm.write("Validation loss %f" % avg_val_loss)

    # Statistik speichern für Auswertung
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Validierung Loss': avg_val_loss,
            'Accuracy': avg_val_accuracy
        }
    )

In [ ]:
import pandas as pd

df_stats = pd.DataFrame(data=training_stats).set_index("epoch")
df_stats

In [ ]:
import matplotlib.pyplot as plt
df_stats.plot()
